In [1]:
import pandas as pd
import re

INPUT_FILE = "message.csv"
OUTPUT_FILE = "transactions.csv"

df = pd.read_csv(INPUT_FILE)

if "message" in df.columns:
    MSG_COL = "message"
elif "body" in df.columns:
    MSG_COL = "body"
else:
    raise ValueError("Could not find a 'message' or 'body' column in messages.csv")


if "sender" in df.columns:
    SENDER_COL = "sender"
elif "address" in df.columns:
    SENDER_COL = "address"
else:
    SENDER_COL = None  



amount_regex = re.compile(r'(?i)(?:rs\.?|inr)\s*([0-9,]+(?:\.[0-9]{1,2})?)')

def is_transaction_message(text: str) -> bool:
    if not isinstance(text, str):
        return False
    t = text.lower()
    keywords = [
        "debited", "credited", "rs", "inr", "upi", "payment",
        "txn", "transaction", "spent", "paid", "purchase", "pos"
    ]
    return any(kw in t for kw in keywords)

def extract_amount(text: str):
    if not isinstance(text, str):
        return None
    m = amount_regex.search(text)
    if not m:
        return None
    amt_str = m.group(1).replace(",", "")
    try:
        return float(amt_str)
    except ValueError:
        return None

def get_txn_type(text: str) -> str:
    if not isinstance(text, str):
        return "unknown"
    t = text.lower()
    if "debited" in t or "spent" in t or "paid" in t or "purchase" in t:
        return "debit"
    if "credited" in t or "received" in t:
        return "credit"
    return "unknown"

def get_merchant(text: str) -> str:
    if not isinstance(text, str):
        return "UNKNOWN"

    lower = text.lower()
    for kw in [" at ", " to ", " in "]:
        if kw in lower:
            parts = text.split(kw, 1)
            if len(parts) > 1:
                rest = parts[1].strip()
                merchant = rest
                merchant = re.sub(r'[^a-zA-Z0-9]', '', merchant)
                return merchant.upper() if merchant else "UNKNOWN"
    return "UNKNOWN"




transactions = []

for idx, row in df.iterrows():
    msg = row.get(MSG_COL, "")

    if not is_transaction_message(msg):
        continue

    amount = extract_amount(msg)
    if amount is None:
        continue  

    txn_type = get_txn_type(msg)
    merchant = get_merchant(msg)

    date = row.get("date", row.get("time", ""))


    sender = row.get(SENDER_COL, "") if SENDER_COL is not None else ""

    transactions.append({
        "date": date,
        "sender": sender,
        "raw_message": msg,
        "amount": amount,
        "txn_type": txn_type,
        "merchant": merchant,
    })

transactions_df = pd.DataFrame(transactions)

transactions_df.to_csv(OUTPUT_FILE, index=False, encoding="utf-8")
print(f"Saved {len(transactions_df)} transactions to {OUTPUT_FILE}")


Saved 252 transactions to transactions.csv
